In [16]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import mlflow
import numpy as np
import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.44.0


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


distributeddeeplearningqmx
deep-learning-challenge
westus2
3df1840f-dd4b-4f54-a831-e20536439b3a


In [17]:
ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


In [21]:
script_folder = 'scripts'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='Hyperdrive_Titanic')


In [4]:
compute_target = ComputeTarget(workspace=ws, name='cpu-cluster')


In [5]:
tf_env = Environment.get(ws, name='AzureML-sklearn-1.0-ubuntu20.04-py38-cpu')


In [7]:
ds_titanic_raw = Dataset.get_by_name(ws, 'titanic_raw')


In [22]:
src = ScriptRunConfig(source_directory=script_folder,
                      script='hyperdrive.py',
                      arguments=['--titanic_raw', ds_titanic_raw.as_named_input('titanic_raw')],
                      compute_target=compute_target,
                      environment=tf_env)

In [42]:
ps = RandomParameterSampling(
    {
        '--solver': choice('liblinear', 'lbfgs'),
        '--penalty': choice('l2'),
        '--tol': choice(0.0001, 0.0002),
        '--max_iter': choice(50, 100)
    }
)

In [43]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


In [47]:
hd_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='training_accuracy_score', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=10,
                             max_concurrent_runs=4)


In [48]:
htr = exp.submit(config=hd_config)


In [49]:
htr

Experiment,Id,Type,Status,Details Page,Docs Page
Hyperdrive_Titanic,HD_e58118d8-bcf1-4cc0-b4ab-ae826ef1a6c8,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation
